In [6]:
import pandas as pd
import requests
import numpy as np
import pymysql
import datetime
from datetime import date


pd.set_option('display.max_rows', None)
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0] 
# 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줌 
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format) 
# 우리가 필요한 것은 회사명과 종목코드이기 때문에 필요없는 column들은 제외해준다. 
code_df = code_df[['회사명', '종목코드']] 
# 한글로된 컬럼명을 영어로 바꿔준다. 
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'}) 
code_df.sort_values(by=['code'], axis=0).head(5)

,name,code
925,동화약품,000020
1504,KR모터스,000040
1510,경방,000050
1530,메리츠화재,000060
37,삼양홀딩스,000070


In [30]:
def get_url(item_name, code_df):
    
    code = code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False)
    url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code.strip())
    print("요청 URL = {}".format(url))
    return url

In [40]:
def get_data(item_name):
    item_name = item_name
    url = get_url(item_name, code_df)
    df = pd.DataFrame()
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36'}
    for page in range(1, 100):
        pg_url = '{url}&page={page}'.format(url=url, page=page)
        pg_url = requests.get(pg_url, headers=headers).text
        df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True)
        
    df = df.dropna()
    #df.head()
    
    df = df.rename(columns = {'날짜' : 'date', '종가' : 'close', '전일비' : 'diff', 
                         '시가' : 'open', '고가' : 'high', '저가' : 'low', '거래량' : 'volume', })
    df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int)
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(by=['date'], ascending=False)
    #df.head()
    df = df.query("date >= '2016-08-01' and date <= '2020-03-12'")
    print(df.head())
    df.head()

In [41]:
item_name='진에어'
get_data(item_name)

요청 URL = http://finance.naver.com/item/sise_day.nhn?code=272450
          date  close  diff   open   high    low  volume
362 2020-03-12  11000  1100  12000  12000  10950  217599
363 2020-03-11  12100   200  12450  12500  12000  120206
364 2020-03-10  12300   250  12150  12750  12150  281820
365 2020-03-09  12550   450  12650  12850  12400  146929
369 2020-03-06  13000   500  13100  13200  12800  172312


In [3]:
# 신라젠의 일자데이터 url 가져오기 
item_name='진에어'
url = get_url(item_name, code_df)

# 일자 데이터를 담을 df라는 DataFrame 정의 
df = pd.DataFrame() 

headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36'}



# 1페이지에서 20페이지의 데이터만 가져오기 
for page in range(1, 100):
    pg_url = '{url}&page={page}'.format(url=url, page=page)
    pg_url= requests.get(pg_url,headers=headers).text
    df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True)
    
df = df.dropna()
df.head()

요청 URL = http://finance.naver.com/item/sise_day.nhn?code=272450


,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.03.05,21950.0,100.0,21700.0,22800.0,21400.0,565826.0
2,2021.03.04,21850.0,1000.0,20950.0,22200.0,20700.0,752268.0
3,2021.03.03,20850.0,100.0,20350.0,21250.0,20150.0,282849.0
4,2021.03.02,20750.0,750.0,21500.0,22000.0,20500.0,472581.0
5,2021.02.26,21500.0,300.0,20700.0,22250.0,20350.0,689271.0


In [4]:
df = df.rename(columns = {'날짜' : 'date', '종가' : 'close', '전일비' : 'diff', 
                         '시가' : 'open', '고가' : 'high', '저가' : 'low', '거래량' : 'volume', })
df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int)
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by=['date'], ascending = False)
df.head()

,date,close,diff,open,high,low,volume
1,2021-03-05,21950,100,21700,22800,21400,565826
2,2021-03-04,21850,1000,20950,22200,20700,752268
3,2021-03-03,20850,100,20350,21250,20150,282849
4,2021-03-02,20750,750,21500,22000,20500,472581
5,2021-02-26,21500,300,20700,22250,20350,689271


In [5]:
df = df.query("date >= '2016-08-01' and date <= '2020-03-12'")
df.head()

,date,close,diff,open,high,low,volume
362,2020-03-12,11000,1100,12000,12000,10950,217599
363,2020-03-11,12100,200,12450,12500,12000,120206
364,2020-03-10,12300,250,12150,12750,12150,281820
365,2020-03-09,12550,450,12650,12850,12400,146929
369,2020-03-06,13000,500,13100,13200,12800,172312


In [24]:
msql = pymysql.connect(
    host='192.168.10.11',
    port=int(3306),
    user='root',
    passwd='1234',
    db='stock_price')
date_time = date.today()
print(date_time)
table_name = 'jin_air'
cursor = msql.cursor()
sql = "insert into test(no, date) values(%s, %s);"
data = (2, date_time)
cursor.execute(sql, data)
msql.commit()
sql = 'select * from test'
cursor.execute(sql)
rows = cursor.fetchall()
print(rows)
msql.close()

2021-03-06
((1, datetime.date(2021, 3, 6)), (2, datetime.date(2021, 3, 6)))
